In [7]:
!pip show torch

Name: torch
Version: 2.1.1+cu121
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.11/dist-packages
Requires: filelock, fsspec, jinja2, networkx, sympy, triton, typing-extensions
Required-by: accelerate, deepspeed, peft, sentence-transformers, timm, torchaudio, torchvision


In [6]:
!pip install transformers==4.44.2
!pip install torch==2.4.1+cu121
!pip install accelerate==0.34.2

ERROR: Could not find a version that satisfies the requirement torch==2.4.1+cu121 (from versions: 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1)
ERROR: No matching distribution found for torch==2.4.1+cu121


In [2]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch

2024-10-03 08:57:00.194348: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-03 08:57:00.194441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-03 08:57:00.272015: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-03 08:57:00.441927: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-03 08:57:02.002489: W tensorflow/compiler/tf2

In [3]:
print(f"Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Using torch 2.1.1+cu121 (NVIDIA RTX A4000)


In [8]:
model_name = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [10]:
messages = [
    {"role": "system", "content": "You are a professional python programmer only respond with code"},
    {"role": "user", "content": "Generate Code for a function creates an sql query to insert data into a table based on some user input"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
print("Tokenized text: ", text)

Tokenized text:  <s>[INST] <<SYS>>
You are a professional python programmer only respond with code
<</SYS>>

Generate Code for a function creates an sql query to insert data into a table based on some user input [/INST]


In [30]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)
print(model_inputs)

{'input_ids': tensor([[    1,     1,   518, 25580, 29962,  3532, 14816, 29903,  6778,    13,
          3492,   526,   263, 10257,  3017, 27922,   871, 10049,   411,   775,
            13, 29966,   829, 14816, 29903,  6778,    13,    13,  5631,   403,
          5920,   363,   263,   740, 10017,   385,  4576,  2346,   304,  4635,
           848,   964,   263,  1591,  2729,   373,   777,  1404,  1881,   518,
         29914, 25580, 29962]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1]], device='cuda:0')}


In [23]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]



Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [25]:
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

[INST] <<SYS>>
You are a professional python programmer only respond with code
<</SYS>>

Generate Code for a function creates an sql query to insert data into a table based on some user input [/INST]  def create_insert_query(table_name, column_names, values):
    query = "INSERT INTO " + table_name + " ("
    for i, column_name in enumerate(column_names):
        if i != 0:
            query += ", "
        query += column_name
    query += ") VALUES ("
    for i, value in enumerate(values):
        if i != 0:
            query += ", "
        query += value
    query += ")"
    return query


In [20]:
pipe = pipeline(
    "text-generation",
    model="codellama/CodeLlama-7b-Instruct-hf",
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [21]:

pipe(messages, max_length=512)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': [{'role': 'system',
    'content': 'You are a professional python programmer only respond with code'},
   {'role': 'user',
    'content': 'Generate Code for a function creates an sql query to insert data into a table based on some user input'},
   {'role': 'assistant',
    'content': '  def create_insert_query(table_name, column_names, values):\n    query = "INSERT INTO " + table_name + " ("\n    query += ", ".join(column_names)\n    query += ") VALUES ("\n    query += ", ".join(["%s"] * len(values))\n    query += ")"\n    return query, values'}]}]